In [3]:
!pip install dlib

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 9.6 MB/s eta 0:00:00a 0:00:01
  Installing build dependencies ... one
  Getting requirements to build wheel ... one
  Preparing metadata (pyproject.toml) ... one
done
  Created wheel for dlib: filename=dlib-20.0.0-cp313-cp313-macosx_12_0_x86_64.whl size=3927704 sha256=46b5406360fc0eaaaaa78f0be7cbd93c66da90cdebc409227248bbd63964cf12
  Stored in directory: /Users/jackduff/Library/Caches/pip/wheels/d4/0d/ef/354663d702c37ab4b234126cfc6b61cfb36eda5ba193613a2c
Successfully built dlib


In [1]:
!pip install firebase-admin


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 2.2 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.3━━━━━━━━━━━━━━━━━━  1/18 [protobuf]
    Uninstalling protobuf-5.29.3:━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  1/18 [protobuf]
      Successfully uninstalled protobuf-5.29.3━━━━━━━━━━━━━━━━  1/18 [protobuf]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18/18 [firebase-admin]m [firebase-admin]restore]s]


In [ ]:
import cv2
import dlib
import numpy as np
from scipy.spatial import distance
import firebase_admin
from firebase_admin import credentials, db
import time

# FIREBASE INITIALIZATION

cred = credentials.Certificate("serviceAccountKey.json")

# Initialize Firebase app
firebase_admin.initialize_app(cred, {
    "databaseURL": "https://driver-drowsiness-system-d98d9-default-rtdb.firebaseio.com/"
})


events_ref = db.reference("drowsiness_events")


# FUNCTION to send event to Firebase
def send_firebase_event():
    event_data = {
        "timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
        "event": "DROWSINESS DETECTED"
    }
    events_ref.push(event_data)
    print("Firebase event uploaded:", event_data)



# EAR FUNCTION
def eye_aspect_ratio(eye):
    A = distance.euclidean(eye[1], eye[5])
    B = distance.euclidean(eye[2], eye[4])
    C = distance.euclidean(eye[0], eye[3])
    ear = (A + B) / (2.0 * C)
    return ear


# DLIB SETUP
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

LEFT_EYE = list(range(36, 42))
RIGHT_EYE = list(range(42, 48))

EAR_THRESHOLD = 0.25
CONSEC_FRAMES = 15
blink_counter = 0
sent_event = False   # prevents spamming Firebase



# VIDEO STREAM
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = detector(gray)

    for face in faces:
        shape = predictor(gray, face)
        landmarks = np.array([[p.x, p.y] for p in shape.parts()])

        left_eye = landmarks[LEFT_EYE]
        right_eye = landmarks[RIGHT_EYE]

        left_ear = eye_aspect_ratio(left_eye)
        right_ear = eye_aspect_ratio(right_eye)
        ear = (left_ear + right_ear) / 2.0

        # Draw eye points
        for (x, y) in np.concatenate((left_eye, right_eye)):
            cv2.circle(frame, (x, y), 2, (0, 255, 0), -1)

        # Drowsiness detection
        if ear < EAR_THRESHOLD:
            blink_counter += 1

            if blink_counter >= CONSEC_FRAMES:
                cv2.putText(frame, "DROWSY!", (50, 50),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                print("DROWSY! [ALERT]")

                if not sent_event:   # avoid multiple uploads
                    send_firebase_event()
                    sent_event = True

        else:
            blink_counter = 0
            sent_event = False  # allow next event
            print("[ALERT OFF]")

    cv2.imshow("EAR Drowsiness Detection", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

# CLEANUP
cap.release()
cv2.destroyAllWindows()


[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
[ALERT OFF]
